In [1]:
import random
import pandas as pd
from sklearn.model_selection import KFold

from helpers_ import (
    train,
    eval,
    calculate_results_for_spectral_clustering_method,
    calculate_average,
    save_parameters_per_method,
    print_results,
)

In [2]:
k_fold = 5
METHODS = [
    "chinese_whispers",
    "correlation_clustering",
    "wsbm",
    "spectral_clustering",
]
NUMBER_OF_WORDS = 24
TARGET_WORDS = [
    "überspannen",
    "Manschette",
    "Fuß",
    "Rezeption",
    "abgebrüht",
    "Dynamik",
    "Engpaß",
    "abbauen",
    "Mißklang",
    "Abgesang",
    "Knotenpunkt",
    "Spielball",
    "zersetzen",
    "Armenhaus",
    "Ohrwurm",
    "Eintagsfliege",
    "Seminar",
    "Sensation",
    "Titel",
    "Schmiere",
    "ausspannen",
    "packen",
    "artikulieren",
    "abdecken",
]
assert len(TARGET_WORDS) == NUMBER_OF_WORDS

In [3]:
def get_fields_to_report_results():
    results_per_method = {}
    parameters_per_method = {}
    for method in METHODS:
        parameters_per_method[method] = {"ari": [], "spr_lscd": []}
        if method != "spectral_clustering":
            results_per_method[method] = {"ari": 0.0, "spr_lscd": 0.0}
        else:
            results_per_method[method] = {
                "avg_ari_silhouette": 0.0,
                "avg_ari_calinski": 0.0,
                "avg_ari_eigengap": 0.0,
                "avg_ari_silhouette_old": 0.0,
                "avg_ari_calinski_old": 0.0,
                "avg_ari_eigengap_old": 0.0,
                "avg_ari_silhouette_new": 0.0,
                "avg_ari_calinski_new": 0.0,
                "avg_ari_eigengap_new": 0.0,
                "spr_lscd_silhouette": 0.0,
                "spr_lscd_calinski": 0.0,
                "spr_lscd_eigengap": 0.0,
            }

    return results_per_method, parameters_per_method

results_per_method, parameters_per_method = get_fields_to_report_results()

In [4]:
kf = KFold(n_splits=k_fold, random_state=32, shuffle=True)
for method in METHODS:
    iteration = 1
    print(f"Method: {method}")

    for train_, test_ in kf.split(TARGET_WORDS):
        best_combination_for_ari, best_combination_for_spr_lscd = train(
            method, train_, TARGET_WORDS
        )
        parameters_per_method[method]["ari"].append(best_combination_for_ari)
        parameters_per_method[method]["spr_lscd"].append(
            best_combination_for_spr_lscd
        )
        ari, spr_lscd = eval(
            method,
            test_,
            best_combination_for_ari,
            best_combination_for_spr_lscd,
            TARGET_WORDS,
        )

        print_results(
            iteration,
            best_combination_for_ari["ari"]
            if "ari" in best_combination_for_ari
            else best_combination_for_ari,
            ari,
            best_combination_for_spr_lscd["spr_lscd"]
            if "spr_lscd" in best_combination_for_spr_lscd
            else best_combination_for_spr_lscd,
            spr_lscd,
        )

        if method != "spectral_clustering":
            results_per_method[method]["ari"] += ari
            results_per_method[method]["spr_lscd"] += spr_lscd
        else:
            ari_and_spr_lscd_calculated_fields = {**ari, **spr_lscd}
            calculate_results_for_spectral_clustering_method(
                results_per_method["spectral_clustering"],
                ari_and_spr_lscd_calculated_fields,
            )

        iteration += 1

calculate_average(results_per_method, METHODS, k_fold)

Method: chinese_whispers


/home/fzamora/miniconda3/envs/ex/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4916: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


  Fold-1:
    dev-ARI: 0.7050238687360847 test-ARI: 0.5459512534781232
    dev-Spr_LSCD: 0.8435533241114941 test-Spr_LSCD: 0.516185401208764

  Fold-2:
    dev-ARI: 0.6978047205487024 test-ARI: 0.6285746099130133
    dev-Spr_LSCD: 0.8729306113538294 test-Spr_LSCD: 0.8999999999999998

  Fold-3:
    dev-ARI: 0.7378210755944455 test-ARI: 0.5200483273426302
    dev-Spr_LSCD: 0.9033817296315558 test-Spr_LSCD: 0.46169025843831935

  Fold-4:
    dev-ARI: 0.7500518174336436 test-ARI: 0.5256599023950035
    dev-Spr_LSCD: 0.9644269704768578 test-Spr_LSCD: 0.5642880936468347

  Fold-5:
    dev-ARI: 0.6918574170981241 test-ARI: 0.695423432761283
    dev-Spr_LSCD: 0.8462707426264467 test-Spr_LSCD: 1.0

Method: correlation_clustering
  Fold-1:
    dev-ARI: 0.7182303089218093 test-ARI: 0.8042905366730719
    dev-Spr_LSCD: 0.8042145329547151 test-Spr_LSCD: 0.9746794344808963

  Fold-2:
    dev-ARI: 0.7356176144611001 test-ARI: 0.7382187756237669
    dev-Spr_LSCD: 0.8127803778557416 test-Spr_LSCD: 0.99

In [5]:
save_parameters_per_method(parameters_per_method, METHODS)

In [6]:
results_per_method, parameters_per_method = get_fields_to_report_results()

In [7]:
for method in METHODS:
    iteration = 1
    print(f"Method: {method}")

    for train_, test_ in kf.split(TARGET_WORDS):
        best_combination_for_ari, best_combination_for_spr_lscd = train(
            method, train_, TARGET_WORDS
        )
        parameters_per_method[method]["ari"].append(best_combination_for_ari)
        parameters_per_method[method]["spr_lscd"].append(
            best_combination_for_spr_lscd
        )
        ari, spr_lscd = eval(
            method,
            test_,
            best_combination_for_ari,
            best_combination_for_spr_lscd,
            TARGET_WORDS,
            exchange_optimized_parameters=True
        )

        print_results(
            iteration,
            best_combination_for_ari["ari"]
            if "ari" in best_combination_for_ari
            else best_combination_for_ari,
            ari,
            best_combination_for_spr_lscd["spr_lscd"]
            if "spr_lscd" in best_combination_for_spr_lscd
            else best_combination_for_spr_lscd,
            spr_lscd,
        )

        if method != "spectral_clustering":
            results_per_method[method]["ari"] += ari
            results_per_method[method]["spr_lscd"] += spr_lscd
        else:
            ari_and_spr_lscd_calculated_fields = {**ari, **spr_lscd}
            calculate_results_for_spectral_clustering_method(
                results_per_method["spectral_clustering"],
                ari_and_spr_lscd_calculated_fields,
            )

        iteration += 1

calculate_average(results_per_method, METHODS, k_fold)

Method: chinese_whispers
  Fold-1:
    dev-ARI: 0.7050238687360847 test-ARI: 0.2914579319203597
    dev-Spr_LSCD: 0.8435533241114941 test-Spr_LSCD: 0.9746794344808963

  Fold-2:
    dev-ARI: 0.6978047205487024 test-ARI: 0.6539381777269202
    dev-Spr_LSCD: 0.8729306113538294 test-Spr_LSCD: 0.8999999999999998

  Fold-3:
    dev-ARI: 0.7378210755944455 test-ARI: 0.5574838134627179
    dev-Spr_LSCD: 0.9033817296315558 test-Spr_LSCD: 0.3

  Fold-4:
    dev-ARI: 0.7500518174336436 test-ARI: 0.404555745334354
    dev-Spr_LSCD: 0.9644269704768578 test-Spr_LSCD: 0.5642880936468347

  Fold-5:
    dev-ARI: 0.6918574170981241 test-ARI: 0.6189588558827837
    dev-Spr_LSCD: 0.8462707426264467 test-Spr_LSCD: 0.39999999999999997

Method: correlation_clustering
  Fold-1:
    dev-ARI: 0.7182303089218093 test-ARI: 0.5995437211318104
    dev-Spr_LSCD: 0.8042145329547151 test-Spr_LSCD: 0.9746794344808963

  Fold-2:
    dev-ARI: 0.7356176144611001 test-ARI: 0.7725347913931706
    dev-Spr_LSCD: 0.8127803778